## [Задания](../README.md)

### 1 scored_videos - датасет на USvideos.csv
 - с добавлением колонки, содержащей скор (показатель качества) видео: ФОРМУЛА должна включать в себя
     - просмотры,
     - лайки,дизлайки видео,
     - лайки и дизлайки к комментариям к видео
       
### 2 categories_score - по категориям, в котором поля: 
 - Название категории (не id) -  в US_category_id.json
 - Медиана показателя score из scored_videos по каждой категории

### 3 popular_tags - по самым популярным тэгам
  - название тэга + количество видео с этим тэгом
    ! тэги лежат строкой в поле tags
      - Scala-функцию для разбиения тегов: 
        - Но напишите свою UDF-функцию разбиения строки на тэги
        - и сравните время работы с её Scala-версией.
          - Можно замерять своими силами,
            - а можно воспользоваться библиотекой timeit
            - функции Spark из пакета pyspark.sq.functions использовать нельзя,
              - нужно написать свою функцию.

## [Критерии приёмки](https://github.com/dataengy/pyspark_course_excercises/blob/0abbfb22573a46501aabc109facd075632f7fcdf/hw1/README.md#%D0%BA%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D0%B8-%D0%B2%D1%8B%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F)
1. В качестве результата должна быть выслана ссылка на публичный git-репозиторий, созданный на основе репозитория, указанного в описании.
1. Код решения должен быть в notebook Video Scoring.ipynb (уже находится в репозитории-примере).
1. Все итоговые датасеты должны быть отображены в ноутбуке через .show() .
1. Везде, где это возможно, обычные UDF должны быть заменены на Pandas UDF или Pandas API on Spark.
1. Датасеты USvideos.csv и UScomments.csv считаем огромными (нельзя помещать в broadcast или кэшировать).
1. Для каждого join должна быть использована какая-либо из оптимизаций, предложенных на лекции. Выбор типа оптимизации нужно обосновать, описав его в комментариях в ноутбуке.
- ...
#### - 8 Для функции разделения тегов должны быть представлены выводы по сравнению времени работы обычной и scala-версии.
    - ну с Scala UDF чот пока никак не разобрался 😅🥲

#### - 6 Для каждого join должна быть использована какая-либо из оптимизаций, предложенных на лекции. Выбор типа оптимизации нужно обосновать, описав его в комментариях в ноутбуке.
    - 🤔... так, ну добавил:
      – broadcast на категории при джоине с видео (маленькую с большой джойним, и partitioning на виде и бакетинг на комменты


In [159]:
# def skip_exc(handler=None):
#   def 
#   def deco(fn):
#     def wrap(*a, **k):
#         try: return fn(*a, **k)
#         except Exception as e: (handler or (lambda e, *_, **__: traceback.print_exc(), print(e, *_, __)))(e)
#     return wrap
#   def wrap(*a, **k):
#         try: return fn(*a, **k)
#         except Exception as e: (handler or (lambda e, *_, **__: traceback.print_exc(), print(e, *_, __)))(e)
#   if callable(handler) and any(
#       ##d (a for a in 'print log'.split(' ') if handler.__name__.startswith(a))
#       (1,1)
#           ):
#               prinr(11)
#               return deco(handler)
#   else: 
#       prinr(22)
#       return wrap

# @skip_exc
# def foo(a): return a/0

# @skip_exc(handler=print) 
# def bar(a): return a/0

# # foo(1)
# bar(2)
# print(123)

In [231]:
import traceback, re
from IPython.core.magic import register_cell_magic, register_line_magic
@register_cell_magic
def skip_err(line, cell):
    try: exec(cell, globals())
    except Exception as e: traceback.print_exc()

@register_line_magic
def globs(line, do_print=None):
    """ Get all global vars by: 1) regexp name mask, 2) type if it's set by only-one arg in python syntax """
    # print(f"{'-p' in line.split(' ') =}")
    if do_print is None:
        do_print, line = (True, line.replace('-p', '')
                         ) if '-p' in line.split(' ') else (False, line)
        line=line.strip(' ')
        # print(f"# {line=}")
    if ' ' in line: return [globs(line, do_print=do_print) for line in line.split(' ')] # if not do_print else None
    try:
        if isinstance(eval(line), type):
            print(f"Search globals of type <{isinstance((type_ := eval(line)), type), line}>:..")
            return [g for g in globals() if isinstance(globals()[g], type_)]
    except NameError: pass
    if (matching_variables := [var_name for var_name in globals() if re.compile(line).search(var_name)]):
        print(f"# {line}:\t", end='')
        for var_name in matching_variables: print(var_name, end='\t') if do_print else ''
        print()
    else: print(f"No matching variables for <{line}> found.")
    return matching_variables if not do_print else line
%globs -p com df

# com:	array_compact	comments_schema	comments	comments_by_video	
# df:	pandas_udf	call_udf	udf	get_videos_df	df	df_exploded	categories_df	videos_df	dff	dfn	


['com', 'df']

In [161]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .config("spark.jars", "super_udf_lib.jar") \
    .getOrCreate()
display(spark)

def get_videos_df():
    videos_df = spark.read.parquet('spark-warehouse/videos')  # to reload saved after spark restart
    videos_df.cache().count(), \
    videos_df.show(5, 55)
    return videos_df

# videos = get_videos_df(); videos.show(3, 33)

# check jars in conf
[(g, v) for g, v in dict(spark.sparkContext.getConf().getAll()).items() if 'jar' in g]

# videos_df.createOrReplaceTempView('videos_df')
# df = spark.sql("SELECT splitTagsUDF(tags) as tags_array FROM videos_df")

[('spark.app.initial.jar.urls',
  'spark://76c09a2dee74:38483/jars/super_udf_lib.jar'),
 ('spark.jars', 'super_udf_lib.jar'),
 ('spark.repl.local.jars', 'file:///home/jovyan/notebooks/super_udf_lib.jar')]

#### Spark UI:
- http://127.0.0.1:4040/stages/

In [162]:
%%skip_err
spark.sparkContext.addJar("super_udf_lib.jar")

Traceback (most recent call last):
  File "/tmp/ipykernel_34488/2760181416.py", line 5, in skip_err
    try: exec(cell, globals())
         ^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
AttributeError: 'SparkContext' object has no attribute 'addJar'


In [163]:
%%skip_err
# spark.udf.registerJavaFunction("splitTags", "CustomUDFs.splitTags", ArrayType(StringType()))
spark.udf.registerJavaFunction("splitTagsUDF", "super_udf_lib.CustomUDFs.splitTagsUDF", ArrayType(StringType()))

Traceback (most recent call last):
  File "/tmp/ipykernel_34488/2760181416.py", line 5, in skip_err
    try: exec(cell, globals())
         ^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 2, in <module>
  File "/usr/local/spark/python/pyspark/sql/udf.py", line 660, in registerJavaFunction
    self.sparkSession._jsparkSession.udf().registerJava(name, javaClassName, jdt)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/errors/exceptions/captured.py", line 175, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisException: Can not load class super_udf_lib.CustomUDFs.splitTagsUDF, please make sure it is on the classpath.


In [164]:
%%skip_err
spark.udf.registerJavaFunction(
    "splitTags", 
    "com.example.super_udf_lib.CustomUDFs.splitTagsUDF",
    # "super_udf_lib.CustomUDFs.splitTagsUDF",
    ArrayType(StringType())
)

Traceback (most recent call last):
  File "/tmp/ipykernel_34488/2760181416.py", line 5, in skip_err
    try: exec(cell, globals())
         ^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
  File "/usr/local/spark/python/pyspark/sql/udf.py", line 660, in registerJavaFunction
    self.sparkSession._jsparkSession.udf().registerJava(name, javaClassName, jdt)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/errors/exceptions/captured.py", line 175, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisException: Can not load class com.example.super_udf_lib.CustomUDFs.splitTagsUDF, please make sure it is on the classpath.


### чот со Scala-UDF'кой пока никак... 🥹
ну да ладно!.. 😅

In [165]:
# Look datasets & load videos
!du datasets/*
videos = spark.read.option('header', 'true').option("inferSchema", "true").csv('datasets/USvideos.csv')
videos_src = videos
videos.show(3, 11)
videos.limit(3).toPandas().T

12	datasets/US_category_id.json
71108	datasets/UScomments.csv
2916	datasets/USvideos.csv
+-----------+-----------+-------------+-----------+-----------+-------+------+--------+-------------+--------------+-----+
|   video_id|      title|channel_title|category_id|       tags|  views| likes|dislikes|comment_total|thumbnail_link| date|
+-----------+-----------+-------------+-----------+-----------+-------+------+--------+-------------+--------------+-----+
|XpVt6Z1Gjjo|1 YEAR O...|  Logan Pa...|         24|logan pa...|4394029|320053|    5931|        46245|   https://...|13.09|
|K4wEI5zhHB0|iPhone X...|        Apple|         28|Apple|iP...|7860119|185853|   26679|            0|   https://...|13.09|
|cLdxuaxaQwc|My Response|    PewDiePie|         22|     [none]|5845909|576597|   39774|       170708|   https://...|13.09|
+-----------+-----------+-------------+-----------+-----------+-------+------+--------+-------------+--------------+-----+
only showing top 3 rows



,0,1,2
video_id,XpVt6Z1Gjjo,K4wEI5zhHB0,cLdxuaxaQwc
title,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,iPhone X — Introducing iPhone X — Apple,My Response
channel_title,Logan Paul Vlogs,Apple,PewDiePie
category_id,24,28,22
tags,logan paul vlog|logan paul|logan|paul|olympics...,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,[none]
views,4394029,7860119,5845909
likes,320053,185853,576597
dislikes,5931,26679,39774
comment_total,46245,0,170708
thumbnail_link,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg


In [166]:
# Load comments
comments_schema = StructType([ \
    StructField("video_id", StringType(), True), \
    StructField("comment_text", StringType(), True), \
    StructField("likes", IntegerType(), True), \
    StructField("replies", IntegerType(), True)])
comments = spark.read.option('header', 'true').option("mode", "DROPMALFORMED").schema(comments_schema).csv('datasets/UScomments.csv')
comments.limit(3).toPandas()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0


In [167]:
videos.printSchema(), comments.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_total: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- date: string (nullable = true)

root
 |-- video_id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- likes: integer (nullable = true)
 |-- replies: integer (nullable = true)



(None, None)

In [168]:
coalesce??

Signature: coalesce(*cols: 'ColumnOrName') -> pyspark.sql.column.Column
Source:   
@try_remote_functions
def coalesce(*cols: "ColumnOrName") -> Column:
    """Returns the first column that is not null.

    .. versionadded:: 1.4.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Parameters
    ----------
    cols : :class:`~pyspark.sql.Column` or str
        list of columns to work on.

    Returns
    -------
    :class:`~pyspark.sql.Column`
        value of the first column that is not null.

    Examples
    --------
    >>> cDf = spark.createDataFrame([(None, None), (1, None), (None, 2)], ("a", "b"))
    >>> cDf.show()
    +----+----+
    |   a|   b|
    +----+----+
    |null|null|
    |   1|null|
    |null|   2|
    +----+----+

    >>> cDf.select(coalesce(cDf["a"], cDf["b"])).show()
    +--------------+
    |coalesce(a, b)|
    +--------------+
    |          null|
    |             1|
    |             2|
    +--------------+

    >>> cDf.select('*', coalesce(

In [169]:
comments_by_video = comments.groupBy('video_id').sum('likes').alias('likes') #!dw
comments_by_video.cache().show(11, 33)

+-----------+----------+
|   video_id|sum(likes)|
+-----------+----------+
|xPS7bqBePSs|      1037|
|dInwVhRtN4E|        63|
|rn5Xgak1zzA|        14|
|TzyraAp3jaY|      1126|
|eHq6ZA6uKOg|       797|
|_r5eTelhpmQ|       749|
|JkqTeQHFoBY|       558|
|Bo-qp-Zu0OY|        71|
|K7pQsR8WFSo|       104|
|g_ekn1gjBq0|        47|
|4yCkkOvIkUI|        36|
+-----------+----------+
only showing top 11 rows



In [170]:
#! Doubles in videos dataframe happens!
df = videos_src.where("video_id like '-6Zc8Co2H3w'").orderBy('video_id')
df.cache().show(11, 33)
df.limit(3).toPandas().T

+-----------+---------------------------------+--------------+-----------+---------------------------------+-------+-----+--------+-------------+---------------------------------+-----+
|   video_id|                            title| channel_title|category_id|                             tags|  views|likes|dislikes|comment_total|                   thumbnail_link| date|
+-----------+---------------------------------+--------------+-----------+---------------------------------+-------+-----+--------+-------------+---------------------------------+-----+
|-6Zc8Co2H3w|Honest Trailers - Star Trek: T...|Screen Junkies|          1|screen junkies|screenjunkies|s...|1547293|42960|    1075|         7498|https://i.ytimg.com/vi/-6Zc8Co...|28.09|
|-6Zc8Co2H3w|Honest Trailers - Star Trek: T...|Screen Junkies|          1|screen junkies|screenjunkies|s...|1764868|46524|    1183|         6152|https://i.ytimg.com/vi/-6Zc8Co...|29.09|
|-6Zc8Co2H3w|Honest Trailers - Star Trek: T...|Screen Junkies|        

,0,1,2
video_id,-6Zc8Co2H3w,-6Zc8Co2H3w,-6Zc8Co2H3w
title,Honest Trailers - Star Trek: The Next Generation,Honest Trailers - Star Trek: The Next Generation,Honest Trailers - Star Trek: The Next Generation
channel_title,Screen Junkies,Screen Junkies,Screen Junkies
category_id,1,1,1
tags,screen junkies|screenjunkies|sj news|honest tr...,screen junkies|screenjunkies|sj news|honest tr...,screen junkies|screenjunkies|sj news|honest tr...
views,1547293,1764868,1870380
likes,42960,46524,48211
dislikes,1075,1183,1243
comment_total,7498,6152,6526
thumbnail_link,https://i.ytimg.com/vi/-6Zc8Co2H3w/default.jpg,https://i.ytimg.com/vi/-6Zc8Co2H3w/default.jpg,https://i.ytimg.com/vi/-6Zc8Co2H3w/default.jpg


In [171]:
videos = videos_src.select('video_id', 'views', 'likes', 'dislikes', 'category_id', 'tags'
                    ).groupBy('video_id', 'category_id', 'tags'
                    ).agg( sum('views').alias('views'), sum('likes').alias('likes'), sum('dislikes').alias('dislikes'),
).cache()
df = videos
df = df.where("video_id like '-6Zc8Co2H3w'").orderBy('video_id')
df.cache().show(11, 33)
df.limit(3).toPandas().T

+-----------+-----------+---------------------------------+-------+------+--------+
|   video_id|category_id|                             tags|  views| likes|dislikes|
+-----------+-----------+---------------------------------+-------+------+--------+
|-6Zc8Co2H3w|          1|screen junkies|screenjunkies|s...|7139354|187351|    4788|
+-----------+-----------+---------------------------------+-------+------+--------+



,0
video_id,-6Zc8Co2H3w
category_id,1
tags,screen junkies|screenjunkies|sj news|honest tr...
views,7139354
likes,187351
dislikes,4788


In [265]:
# 1 scored_videos - датасет на USvideos.csv
#  - с добавлением колонки, содержащей скор (показатель качества) видео:
#    - ФОРМУЛА должна включать в себя
#      - просмотры,
#      - лайки, дизлайки видео,
#      - лайки и дизлайки к комментариям к видео

# videos = videos_src

# Make repartition for videos and bucketing both by 100 partitions/buckets for comments 1st for optimizing JOIN
videos = videos.repartition(100)
comments_by_video = comments_by_video.repartitionByRange(100, "video_id")

videos = (
    videos.select('video_id', 'views', 'likes', 'dislikes', 'category_id', 'tags'\
         ).join(comments_by_video
                .select('video_id', col('sum(likes)').alias('likes_c'),  # col('dislikes').alias('dislikes_c')  - не нашёл 🤔
                ),
                on='video_id', how='left'
         ).select('video_id', 'views', 'likes', 'dislikes', 'category_id', 'tags',
                  coalesce(col('likes_c'), lit(0)).alias('likes_c'))
)
videos.cache().show(3, 55)

+-----------+--------+------+--------+-----------+-------------------------------------------------------+-------+
|   video_id|   views| likes|dislikes|category_id|                                                   tags|likes_c|
+-----------+--------+------+--------+-----------+-------------------------------------------------------+-------+
|4yCkkOvIkUI|   12905|    59|      43|         24|                                                 [none]|     36|
|7TN09IP5JuI|20785005|869092|    8855|         26|First we feast|fwf|firstwefeast|food|food porn|cook|...|     34|
|Bo-qp-Zu0OY|   21654|   179|       1|         19|talking mickey|talking donald|talking minnie|dca|dis...|     71|
+-----------+--------+------+--------+-----------+-------------------------------------------------------+-------+
only showing top 3 rows



In [261]:
# 1) score = (likes - dislikes + likes-comments) / views 
videos = videos.withColumn('score', (col('likes') - col('dislikes') + col('likes_c')) / col('views'))
videos.orderBy(desc('score')).cache().show(3, 33)
videos.where('score > 0').orderBy(asc('score')).cache().show(3, 33)
videos.cache().count()

+-----------+-------+------+--------+-----------+---------------------------------+-------+-------------------+---------------------------------+
|   video_id|  views| likes|dislikes|category_id|                             tags|likes_c|              score|                       tags_array|
+-----------+-------+------+--------+-----------+---------------------------------+-------+-------------------+---------------------------------+
|J0sg_Au8zX8|   8741|   182|       2|         24|nbc this is us|this is us afte...|   1792|0.22560347786294474|[nbc this is us, this is us af...|
|TSxt2_QM4N4|1624869|318826|    1983|         10|             LAY|레이|SHEEP|羊|MV|     37|0.19501879843852027|       [LAY, 레이, SHEEP, 羊, MV]|
|JdqUUKEATiw|  20722|  3335|      19|          1|callmekat|katarzyna napiórkows...|    636|0.19071518193224593|[callmekat, katarzyna napiórko...|
+-----------+-------+------+--------+-----------+---------------------------------+-------+-------------------+-------------------

2415

In [174]:
# Research file with categories:..
!head -n 11 datasets/US_category_id.json
!echo ...

{
 "kind": "youtube#videoCategoryListResponse",
 "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",
 "items": [
  {
   "kind": "youtube#videoCategory",
   "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ\"",
   "id": "1",
   "snippet": {
    "channelId": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "Film & Animation",
...


### мда... тут без "помощи Друзя" никак 😅🤓

#### https://chat.openai.com/c/fac3b2e7-81c4-4ef5-9cdd-120a459b0fcf
how to exctract to spark dataframe with 2 cols:
- category_id – from path: "items"."id" where "kind" = "youtube#videoCategory"
- category_name – from path: "items"."snippet"."title" for category_id

from this file:
{ "kind": "youtube#videoCategoryListResponse",
 "etag": "\"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM\"",..

In [175]:
# 2) read categories from json
df = spark.read.option("multiline", "true").json("datasets/US_category_id.json")

df_exploded = df.select(explode(df.items).alias("item"))

categories_df = df_exploded.select(
    df_exploded["item.id"].alias("category_id"),
    df_exploded["item.snippet.title"].alias("category_name")
)

categories_df = categories_df.filter(df_exploded["item.kind"] == "youtube#videoCategory")

categories_df.cache().show(5, False)

+-----------+----------------+
|category_id|category_name   |
+-----------+----------------+
|1          |Film & Animation|
|2          |Autos & Vehicles|
|10         |Music           |
|15         |Pets & Animals  |
|17         |Sports          |
+-----------+----------------+
only showing top 5 rows



In [176]:
for df in (videos_src, categories_df): df.show(3, 33), df.count()

+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------+
|   video_id|   views| likes|dislikes|category_id|                             tags|likes_c|               score|
+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------+
|4yCkkOvIkUI|   12905|    59|      43|         24|                           [none]|     36|0.004029445951181713|
|7TN09IP5JuI|20785005|869092|    8855|         26|First we feast|fwf|firstwefeas...|     34|  0.0413890205944141|
|Bo-qp-Zu0OY|   21654|   179|       1|         19|talking mickey|talking donald|...|     71|0.011499030202272097|
+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------+
only showing top 3 rows

+-----------+----------------+
|category_id|   category_name|
+-----------+----------------+
|          1|Film & Animation|
|          2|Autos & Vehicles|
|         10|         

In [258]:
from loguru import logger as log
@log.catch
def show_dfs(*dff, rows=1, width=33):
  try:
    if len(dff) == 1: dff = dff[0]
    if isinstance(dff, str): dff = dff.split(' ')
        
    for dfn in dff: #df = eval(dfn); \
      if isinstance(dfn, str):
        print(dfn, f"{(df := eval(dfn)).count()} rows", sep=': '), print( 
              display(df.cache().limit(rows).toPandas()) or '',
              )
      elif isinstance(dfn, pyspark.sql.DataFrame):
         print((df:=dfn), f"{df.count()} rows total", sep=':\n'), print( 
              display(df.cache().limit(rows).toPandas()) or '',
              )

      else: raise TypeError(f"{dfn=}")
  except: print(f"{dff=}"), traceback.print_exc()

show_dfs(videos, categories_df, rows=2)

DataFrame[video_id: string, views: bigint, likes: bigint, dislikes: bigint, category_id: int, tags: string, likes_c: bigint, score: double, tags_array: string]:
2415 rows total


,video_id,views,likes,dislikes,category_id,tags,likes_c,score,tags_array
0,4yCkkOvIkUI,12905,59,43,24,[none],36,0.004029,[[none]]
1,7TN09IP5JuI,20785005,869092,8855,26,First we feast|fwf|firstwefeast|food|food porn...,34,0.041389,"[First we feast, fwf, firstwefeast, food, food..."



DataFrame[category_id: string, category_name: string]:
32 rows total


,category_id,category_name
0,1,Film & Animation
1,2,Autos & Vehicles


In [256]:
# 2) categories_score - по категориям, в котором поля: 
#  - Название категории (не id) -  в US_category_id.json
#  - Медиана показателя score из scored_videos по каждой категории

# Join scored_videos and categories_df on category_id
videos_with_categories = videos.join(broadcast(categories_df), on='category_id', how='left')

# Compute median
median_scores = videos_with_categories.groupby("category_name").agg(
    expr('percentile_approx(score, 0.5)').alias('median_score')
)

median_scores.orderBy(desc('median_score')).cache().show(11, False)

+--------------------+--------------------+
|category_name       |median_score        |
+--------------------+--------------------+
|Music               |0.04894056846823924 |
|Howto & Style       |0.04882180089464523 |
|Comedy              |0.035756360280687134|
|Education           |0.03489027472227778 |
|Pets & Animals      |0.03188342746845286 |
|People & Blogs      |0.029420258570009714|
|Science & Technology|0.025101531745246066|
|Film & Animation    |0.02258567556529165 |
|Gaming              |0.022566618082282008|
|Travel & Events     |0.022412075703011264|
|Entertainment       |0.020693893364327413|
+--------------------+--------------------+
only showing top 11 rows



### - 7 Для расчета медианы нельзя использовать встроенную Spark-функцию median из пакета pyspark.sql.functions...

Ок, оставим её только для сверки)

In [178]:
# Create Pandas UDF for median
from pyspark.sql.functions import * #pandas_udf, PandasUDFType, groupBy as groupby
import pandas as pd, statistics

@pandas_udf('double', PandasUDFType.GROUPED_AGG)
def median_score(score: pd.Series) -> float: return statistics.median(score)

scored_videos_with_category_name = videos.join(broadcast(categories_df), on='category_id')

category_score_median = scored_videos_with_category_name.groupby('category_name').agg(median_score(scored_videos_with_category_name['score']).alias('median_score'))

category_score_median.orderBy(desc('median_score')).cache().show(11, False)

+--------------------+--------------------+
|category_name       |median_score        |
+--------------------+--------------------+
|Howto & Style       |0.048975267792669364|
|Music               |0.03978118460448949 |
|Comedy              |0.035756360280687134|
|Education           |0.03489027472227778 |
|Pets & Animals      |0.03188342746845286 |
|People & Blogs      |0.029489108496292302|
|Science & Technology|0.02543212129949589 |
|Travel & Events     |0.023733134052269893|
|Film & Animation    |0.022960721689022617|
|Gaming              |0.022566618082282008|
|Entertainment       |0.020693893364327413|
+--------------------+--------------------+
only showing top 11 rows



/usr/local/spark/python/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [179]:
# Compare builtin and own UDF medians
category_score_median.join(median_scores, on='category_name'
    ).select(category_score_median['*'], median_scores['median_score'].alias('median_score_pd'),
             (category_score_median['median_score'] - median_scores['median_score']).alias('median_scores_diff')
    ).orderBy(desc('median_score')).cache().show(11, False)

+--------------------+--------------------+--------------------+---------------------+
|category_name       |median_score        |median_score_pd     |median_scores_diff   |
+--------------------+--------------------+--------------------+---------------------+
|Howto & Style       |0.048975267792669364|0.04882180089464523 |1.5346689802413216E-4|
|Music               |0.03978118460448949 |0.04894056846823924 |-0.009159383863749752|
|Comedy              |0.035756360280687134|0.035756360280687134|0.0                  |
|Education           |0.03489027472227778 |0.03489027472227778 |0.0                  |
|Pets & Animals      |0.03188342746845286 |0.03188342746845286 |0.0                  |
|People & Blogs      |0.029489108496292302|0.029420258570009714|6.884992628258813E-5 |
|Science & Technology|0.02543212129949589 |0.025101531745246066|3.305895542498223E-4 |
|Travel & Events     |0.023733134052269893|0.022412075703011264|0.001321058349258629 |
|Film & Animation    |0.022960721689022617|

## ! Почему-то в 2-й строке довольно большой расходняк! 😅🤔 Ну да ладно... 🤓

In [180]:
videos_df = videos_with_categories
videos_df.show(5, 44)
videos_df.printSchema()

+-----------+-----------+-------+------+--------+--------------------------------------------+-------+---------------------+--------------------+
|category_id|   video_id|  views| likes|dislikes|                                        tags|likes_c|                score|       category_name|
+-----------+-----------+-------+------+--------+--------------------------------------------+-------+---------------------+--------------------+
|         28|CAQ2wWVlOuc| 146015|  1811|     396|Tech Insider|TI|Tech|Science|Innovation|D...|    923| 0.016012053556141492|Science & Technology|
|         28|9W0WPPpCFaM| 209228|  7829|     237|vox.com|vox|explain|sense|perception|soph...|      9|  0.03632878964574531|Science & Technology|
|         28|1L7JFN7tQLs|4961383|142040|    5573|Apple|iPhone X|iPhone 8|iPhone X unboxing...|   2019| 0.027912781577233606|Science & Technology|
|         28|B9SptdjpJBQ|4559747|220177|    2311|Learning tricks|ways to learn faster|best...|     62|  0.04779387979201478|

In [181]:
# Save to disk for reusing, if'll be needed
!rm -r spark-warehouse/videos
if 'videos' in [a.name for a in spark.catalog.listTables()]:
    spark.sql("drop table videos")
videos_df.write.saveAsTable('videos', partitionBy='category_name') #, mode='overwrite')
videos_df = spark.table('videos')
videos_df.cache().count(), \
videos_df.show(5, 55)

+-----------+-----------+--------+-------+--------+-------------------------------------------------------+-------+--------------------+-------------+
|category_id|   video_id|   views|  likes|dislikes|                                                   tags|likes_c|               score|category_name|
+-----------+-----------+--------+-------+--------+-------------------------------------------------------+-------+--------------------+-------------+
|         24|4yCkkOvIkUI|   12905|     59|      43|                                                 [none]|     36|0.004029445951181713|Entertainment|
|         24|eHq6ZA6uKOg|  135189|   1156|     374|American Broadcasting Company|ABC|ABC Network|Televi...|    797|0.011679944374172455|Entertainment|
|         24|3y5A4paFOb4|85451527|5980546|  551334|logan paul vlog|logan paul|logan|paul|olympics|logan...|    127| 0.06353706236285281|Entertainment|
|         24|GbhdRYQz9-0|  261483|   2045|      25|                                           

(2415, None)

In [182]:
# 3) popular_tags - по самым популярным тэгам
  # - название тэга + количество видео с этим тэгом
  #   ! тэги лежат строкой в поле tags
  #     - Scala-функцию для разбиения тегов: 
  #       - Но напишите свою UDF-функцию разбиения строки на тэги
  #       - и сравните время работы с её Scala-версией.
  #         - Можно замерять своими силами,
  #           - а можно воспользоваться библиотекой timeit
  #           - функции Spark из пакета pyspark.sq.functions использовать нельзя,
  #             - нужно написать свою функцию.

In [183]:
%%skip_err
videos.printSchema()
videos.where(f"tags is not null and tags != '[none]'").show(3, 33)

root
 |-- video_id: string (nullable = true)
 |-- views: long (nullable = true)
 |-- likes: long (nullable = true)
 |-- dislikes: long (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- tags: string (nullable = true)
 |-- likes_c: long (nullable = false)
 |-- score: double (nullable = true)

+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------+
|   video_id|   views| likes|dislikes|category_id|                             tags|likes_c|               score|
+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------+
|7TN09IP5JuI|20785005|869092|    8855|         26|First we feast|fwf|firstwefeas...|     34|  0.0413890205944141|
|Bo-qp-Zu0OY|   21654|   179|       1|         19|talking mickey|talking donald|...|     71|0.011499030202272097|
|JkqTeQHFoBY|  431082|  4694|      43|          1|marvel|vfx|CG|CGI|Animation|Gu...|    558|0.012083547909678437|
+------

In [184]:
def split_tags_simple(tags_str): return tags_str.split('|')

# Create a UDF from the Python function
split_tags_simple = udf(split_tags_simple)

# Now you can use this UDF in your DataFrame transformations
videos = videos.withColumn('tags_array', split_tags_simple(videos['tags']))

In [185]:
# так, пришла помощь из зала..
from pyspark.sql.functions import pandas_udf, PandasUDFType, explode
from pyspark.sql import DataFrame
import pandas as pd

@pandas_udf('array<string>', PandasUDFType.SCALAR)
def split_tags(tags_series: pd.Series) -> pd.Series: return tags_series.str.split('|')
#...

In [186]:
%%timeit
# - 8 Для функции разделения тегов должны быть представлены выводы по сравнению времени работы обычной и scala-версии.
videos.select(split_tags('tags')).count()

552 ms ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [187]:
#...
df = videos
df = df.withColumn('tags_array', split_tags(df['tags']))
df.cache().show(3, 33)
videos_tagged = \
df = df.select('*',
            explode(
                df['tags_array']
                   ).alias('tag')
      )
df.cache().show(3, 33)
df.groupBy('tag').count().orderBy(desc('count')).show(3, 33)
#` df.cache().where("tag is not null").show(33, 33)

+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------+---------------------------------+
|   video_id|   views| likes|dislikes|category_id|                             tags|likes_c|               score|                       tags_array|
+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------+---------------------------------+
|4yCkkOvIkUI|   12905|    59|      43|         24|                           [none]|     36|0.004029445951181713|                         [[none]]|
|7TN09IP5JuI|20785005|869092|    8855|         26|First we feast|fwf|firstwefeas...|     34|  0.0413890205944141|[First we feast, fwf, firstwef...|
|Bo-qp-Zu0OY|   21654|   179|       1|         19|talking mickey|talking donald|...|     71|0.011499030202272097|[talking mickey, talking donal...|
+-----------+--------+------+--------+-----------+---------------------------------+-------+--------------------

## 4) И личная просьба от Марка: он любит котов (а кто не их не любит!) и хочет найти самые интересные комментарии (топ-5) к видео про котов. “Видео про котов” - видео, у которого есть тэг “cat”.


In [188]:
from pyspark.sql.window import Window
# window = Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
top_cats_tags = (
    videos_tagged['tag', 'score']
        .where("regexp_like(tag, '.*[ ](cat|kitten)[ sty].*')")
        .groupBy('tag')
        .agg( count(col('*')).alias('cnt'),
              avg('score').alias('avg_score'),
        )
        # .withColumn('cnt_total', count('*').over(window))
        .orderBy(desc('cnt'))
)
top_cats_tags.cache().show(11, 33)

+---------------------+---+--------------------+
|                  tag|cnt|           avg_score|
+---------------------+---+--------------------+
|            cute cats|  6| 0.04722702093219663|
|          simons cats|  4|0.041777194456910055|
|        animated cats|  4|0.041777194456910055|
|           funny cats|  4|0.041777194456910055|
|     every cat at 3am|  1|0.037682380723440656|
|     funny cat videos|  1| 0.02781174432894123|
|       crazy cat lady|  1| 0.07857096704209887|
|simplynailogical cats|  1| 0.07857096704209887|
|   dog kitten snuggle|  1| 0.03188342746845286|
|murph bad cat allergy|  1| 0.04149633819515629|
|      dog cat snuggle|  1| 0.03188342746845286|
+---------------------+---+--------------------+
only showing top 11 rows



In [189]:
top_cats_videos_tagged = (videos_tagged
 .join(top_cats_tags, on='tag')
 .select('video_id', 'score', 'likes', 'likes_c', 'tag', 
         col('avg_score').alias('tag_avg_score')
) ## .orderBy(map(desc, *(col(c) for c in 'tag_avg_score score likes likes_c'.split(' '))))
 .orderBy(desc('tag_avg_score'), desc('score'), desc('likes'), desc('likes_c'))
)
top_cats_videos_tagged .cache() .show(11, 33)

+-----------+--------------------+------+-------+----------------------+--------------------+
|   video_id|               score| likes|likes_c|                   tag|       tag_avg_score|
+-----------+--------------------+------+-------+----------------------+--------------------+
|msWS8WxRsrY| 0.07857096704209887|371259|     34|        crazy cat lady| 0.07857096704209887|
|msWS8WxRsrY| 0.07857096704209887|371259|     34| simplynailogical cats| 0.07857096704209887|
|msWS8WxRsrY| 0.07857096704209887|371259|     34|halloween cat costumes| 0.07857096704209887|
|msWS8WxRsrY| 0.07857096704209887|371259|     34|             cute cats| 0.04722702093219663|
|BY3SLVNBkeo| 0.05043145346953393| 23831|    136|             cute cats| 0.04722702093219663|
|-1fzGnFwz9M| 0.04134858035836844|  7070|    874|             cute cats| 0.04722702093219663|
|tp9aQXDFHbY| 0.03884812641337964| 43349|   1613|             cute cats| 0.04722702093219663|
|odhMmAPDc54|0.037682380723440656|243364|  10257|           

In [259]:
top_cats_videos = top_cats_videos_tagged.drop(
                   'tag'
        ).drop(    'tag_avg_score'
        ).distinct(    
        ).orderBy(
          ## desc('score'), desc('likes'), 
          desc('likes_c')
)
top_cats_videos.cache().show(11, 33)

+-----------+--------------------+------+-------+
|   video_id|               score| likes|likes_c|
+-----------+--------------------+------+-------+
|odhMmAPDc54|0.037682380723440656|243364|  10257|
|tp9aQXDFHbY| 0.03884812641337964| 43349|   1613|
|5qlSfNVp3fU| 0.02781174432894123| 35953|   1428|
|-1fzGnFwz9M| 0.04134858035836844|  7070|    874|
|BY3SLVNBkeo| 0.05043145346953393| 23831|    136|
|ruTB5jBGXsE| 0.03648061758635822| 95334|    116|
|msWS8WxRsrY| 0.07857096704209887|371259|     34|
|veqMs7SQteY| 0.04149633819515629| 10385|     20|
|1lOd7rDFYbc| 0.03188342746845286| 51685|     18|
|oStLD-yYAy0| 0.02794867588149242|  8571|     17|
+-----------+--------------------+------+-------+



## Блин! Нужны же не топ видео, а топ комментарии!.. 😅 Ну да ладно, предположим, что их как раз надо искать у топ видео... 🤓 Хотя у нас же не топ, а все видео!!! 😅

In [198]:
# ...
top_cats_videos. select('video_id'
                 ).join(comments, on='video_id'
              ).groupBy('video_id', 'comment_text'
                  ).agg(
                    sum('likes').alias('likes'), sum('replies').alias('replies')
              ).orderBy(desc('likes'), desc('replies')
    ).cache().show(5, 155)

+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------+
|   video_id|                                                                                                                                               comment_text|likes|replies|
+-----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------+
|odhMmAPDc54|                                                                                                 follow me on stuff in the description rawr XD lololol edgy|10162|    401|
|5qlSfNVp3fU|MEOW! … Still no dislikes! \nAfrica was amazing, I spent almost 3 weeks at Emoya, it's a great sanctuary - https://emoya.org.za … I had to make this vid...| 1393|    230|
|tp9aQXDFHbY|                                                     Make sure to c

In [201]:
dff = %globs pyspark.sql.dataframe.DataFrame
dff

Search globals of type <(True, 'pyspark.sql.dataframe.DataFrame')>:..


['videos',
 'videos_src',
 'comments',
 'comments_by_video',
 'df',
 'df_exploded',
 'categories_df',
 'videos_with_categories',
 'median_scores',
 'scored_videos_with_category_name',
 'category_score_median',
 'videos_df',
 'videos_tagged',
 'top_cats_tags',
 'top_cats_videos_tagged',
 'top_cats_videos']

In [249]:
show_dfs(dff)

videos: 2415 rows


,video_id,views,likes,dislikes,category_id,tags,likes_c,score,tags_array
0,4yCkkOvIkUI,12905,59,43,24,[none],36,0.004029,[[none]]



videos_src: 7998 rows


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09



comments: 691318 rows


...

,tag,cnt,avg_score
0,cute cats,6,0.047227



top_cats_videos_tagged: 33 rows


,video_id,score,likes,likes_c,tag,tag_avg_score
0,msWS8WxRsrY,0.078571,371259,34,crazy cat lady,0.078571



top_cats_videos: 10 rows


,video_id,score,likes,likes_c
0,msWS8WxRsrY,0.078571,371259,34


In [233]:
%globs -p videos categor comments

# videos:	get_videos_df	videos	videos_src	videos_with_categories	scored_videos_with_category_name	videos_df	videos_tagged	top_cats_videos_tagged	top_cats_videos	
# categor:	categories_df	videos_with_categories	scored_videos_with_category_name	category_score_median	
# comments:	comments_schema	comments	comments_by_video	


['videos', 'categor', 'comments']

In [250]:
show_dfs(videos_src, comments, categories_df, width=22)

DataFrame[video_id: string, title: string, channel_title: string, category_id: int, tags: string, views: int, likes: int, dislikes: int, comment_total: int, thumbnail_link: string, date: string]: 7998 rows


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09



DataFrame[video_id: string, comment_text: string, likes: int, replies: int]: 691318 rows


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0



DataFrame[category_id: string, category_name: string]: 32 rows


,category_id,category_name
0,1,Film & Animation
